In [2]:
%pylab inline
import xml.sax.saxutils as saxutils
import re
import pandas as pd
from bs4 import BeautifulSoup
import codecs


Populating the interactive namespace from numpy and matplotlib


In [3]:
#General constants 

# Newsline folder and format
data_folder = '/Users/xiuli/Downloads/reuters21578/' 

sgml_number_of_files = 22
sgml_file_name_template = 'reut2-NNN.sgm'

In [4]:
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
#train_Ye=to_category_vector(train_Y) 

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
stop_words = set(stopwords.words('english'))
#stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)


In [6]:
document_X = {}
document_Y = {}

document_id=[]
document_body=[]
document_train_test=[]
category=[]
document_X=pd.DataFrame()

for i in range(sgml_number_of_files):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
    print('Reading file: %s' % file_name)
    
    with codecs.open(data_folder + file_name, 'r', encoding='utf-8',
                 errors='ignore') as file:
        content = BeautifulSoup(file.read().lower())      
        for newsline in content('reuters'):
                      
            # News-line Id
            document_id.append(newsline['newid'])
            
            # News-line text
            document_body.append(unescape(strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')))
#            document_body.append(newsline.find("title").text)
            document_train_test.append(strip_tags(str(newsline.attrs["lewissplit"]) ))
            #for c in newsline.find("topics").findAll('d'):
                #category.append(c.text)
            category.append([item.text for item in newsline.find("topics").findAll('d')])
            
document_X['document_id'] = document_id
document_X['document_body'] = document_body
#document_X['document_body'] = document_X['document_body'].str.lower()
document_X['document_body'] = document_X['document_body'].apply(cleanHtml)
document_X['document_body'] = document_X['document_body'].apply(cleanPunc)
document_X['document_body'] = document_X['document_body'].apply(keepAlpha)
document_X['document_body'] = document_X['document_body'].apply(removeStopWords)
document_X['train_test_label'] = document_train_test
document_X['category'] = category

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm
Reading file: reut2-021.sgm


In [38]:
train_X=document_X[(document_X['train_test_label']=='train') & (document_X['category'].astype(str)!='[]') & (document_X['document_body'].astype(str)!='None')]['document_body'].values.tolist()
test_X=document_X[(document_X['train_test_label']=='test') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['document_body'].values.tolist()

train_Y=document_X[(document_X['train_test_label']=='train') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['category'].values.tolist()
test_Y=document_X[(document_X['train_test_label']=='test') & (document_X['category'].astype(str)!='[]')& (document_X['document_body'].astype(str)!='None')]['category'].values.tolist()


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_X)
vectorizer.fit(test_X)
x_train = vectorizer.transform(train_X)
x_test = vectorizer.transform(test_X)


# Transform multilabel labels
mlb = MultiLabelBinarizer()

train_labels = mlb.fit_transform(train_Y)
test_labels = mlb.transform(test_Y)

In [42]:
len(train_X)

7068

In [40]:
from skmultilearn.adapt import  
classifier_new = MLkNN(k=10)
# train
classifier_new.fit(x_train, train_labels)

MLkNN()

In [41]:
# predict
predictions_new = classifier_new.predict(x_test)

# accuracy
from sklearn.metrics import accuracy_score
print("Accuracy = ",accuracy_score(test_labels,predictions_new))
print("\n")

Accuracy =  0.7686703096539163


